In [1]:

import pandas as pd
file_path = r'D:\GITHUB\SalaryEstimator\documentation\data\df_encoded.json'
df = pd.read_json(file_path, lines=True)



In [2]:
a = df.columns.tolist()  # Display all column names

In [3]:
# 选择指定的两列
cols = ['Cost_Index', 'GDP_per_capita']

# 选择所有以 Country_ 开头的列
country_cols = [col for col in df.columns if col.startswith('Country_')]

# 合并所有需要的列
selected_cols = cols + country_cols

# 取出这些列

df_selected = df[selected_cols]

df_selected = df[selected_cols].drop_duplicates()

# 找到每行对应的国家名
df_selected['Country'] = df_selected[country_cols].idxmax(axis=1).str.replace('Country_', '')

# 如果有多于一个国家为1，可以用下面的方式得到所有国家名列表
# df_selected['Country'] = df_selected[country_cols].apply(lambda x: ','.join(x.index[x==1]).replace('Country_', ''), axis=1)

In [4]:
df_selected[["Country", 'Cost_Index', 'GDP_per_capita']] 

,Country,Cost_Index,GDP_per_capita
0,Pakistan,10.299143,1484.749859
1,Türkiye,13.506102,15473.287264
2,United States,62.048253,85809.900385
3,United Kingdom,64.380714,52636.786594
5,Greece,41.494848,24752.106840
...,...,...,...
12322,Cambodia,23.800000,2627.879697
12840,Botswana,32.915000,7695.242064
12918,Cameroon,20.652857,1762.368351
14701,Suriname,80.000000,7430.702192


In [28]:
# df_selected[["Country", "Cost_Index", "GDP_per_capita"]].to_json("selected_country.json", orient="records", force_ascii=False)

In [6]:
import json
with open('D:\\GITHUB\\SalaryEstimator\\code\\web\\selected_country.json', 'r', encoding='utf-8') as f:
    country_data = json.load(f)

    # country_map = {item['Country']: item for item in country_data}
    

In [7]:
country_map = {item['Country']: item for item in country_data}

In [9]:
print(country_map.get('United States', {}).get('GDP_per_capita', 0))
print(country_map.get('United States', {}).get('Cost_Index', 0))

85809.9003846356
62.0482529475


In [23]:
import joblib
import pandas as pd
import numpy as np
import os
# Load model
model_path = os.path.join(os.path.abspath(), 'lr_model.pkl')

def map_experience_level(years):
    if pd.isna(years):
        return np.nan
    elif years < 2:
        return 'Beginner'
    elif years < 5:
        return 'Intermediate'
    elif years < 10:
        return 'Advanced'
    else:
        return 'Expert'

# User input for numerical features
user_input = {
    'YearsCodePro': 10,
    'Age': 30,
    'WorkExp': 7,
    'GDP_per_capita': 85809.9003846356,
    'Cost_Index': 62.0482529475
}

# Dictionary selections for one-hot encoded features
selections = {
    'EdLevel': 'Bachelor\'s degree (B.A., B.S., B.Eng., etc.)',
    'RemoteWork': 'Remote',
    'OrgSize': '100 to 499 employees',
     'Country': 'United States',
    #  'Country': 'Canada',
    'DevType': 'Developer, full-stack',
    'Employment': 'Employed, full-time',
    'Languages': ['Python', 'JavaScript']
}

# Create DataFrame with all zeros
df = pd.DataFrame(0, index=[0], columns=model.feature_names_in_)

# Fill numerical features
for key, value in user_input.items():
    if key in df.columns:
        df[key] = value

# Calculate ExperienceLevel based on YearsCodePro
experience_level = map_experience_level(user_input['YearsCodePro'])

# Fill one-hot encoded features
for feature, selection in selections.items():
    if feature == 'Languages':
        # Handle multiple language selection
        for lang in selection:
            lang_col = f'Lang_{lang}'
            if lang_col in df.columns:
                df[lang_col] = 1
    else:
        # Handle single selection
        col_name = f'{feature}_{selection}'
        if col_name in df.columns:
            df[col_name] = 1

# Handle ExperienceLevel separately since it's calculated automatically
if experience_level is not None:
    exp_col = f'ExperienceLevel_{experience_level}'
    if exp_col in df.columns:
        df[exp_col] = 1

# Make prediction
prediction = model.predict(df)[0]
actual_salary = np.power(10, prediction)

print(f"Predicted log salary: {prediction:.4f}")
print(f"Predicted annual salary: ${actual_salary:,.2f}")
print(f"Experience Level (calculated): {experience_level}")



TypeError: abspath() missing 1 required positional argument: 'path'